# Submitting Cluster Jobs

## Overview

### Questions

* How do I submit workflows on HPC resources? 
* How can I combine many simulations into a single cluster job?

### Objectives

* Show how to structure the workflow in `project.py` and use **signac-flow's command line interface**.

## Boilerplate code

## Command line interface

In the previous sections, this tutorial uses **signac-flow's** Python API within a Jupyter notebook.
**signac-flow** also has a **command line interface**.
To use it, place the entire worfklow in a Python file and add a `__main__` entry point that calls `FlowProject().main()`.
Here is the file `project.py` with **workflow step** code from the previous two tutorial sections and the main entry point.

In [1]:
%pycat project.py

import hoomd
import signac
import flow

import math


N_EQUIL_STEPS=200000
WALLTIME_LIMIT=30

def create_simulation(job):
    cpu = hoomd.device.CPU()
    sim = hoomd.Simulation(device=cpu, seed=job.statepoint.seed)
    mc = hoomd.hpmc.integrate.ConvexPolyhedron()
    mc.shape['octahedron'] = dict(vertices=[
        (-0.5, 0, 0),
        (0.5, 0, 0),
        (0, -0.5, 0),
        (0, 0.5, 0),
        (0, 0, -0.5),
        (0, 0, 0.5),
    ])
    sim.operations.integrator = mc
    
    return sim

@flow.FlowProject.operation
@flow.FlowProject.pre(lambda job: job.document.get('initialized', False))
@flow.FlowProject.post(lambda job: job.document.get('randomized', False))
def randomize(job):
    sim = create_simulation(job)
    sim.create_state_from_gsd(filename=job.fn('lattice.gsd'))
    sim.run(10e3)
    hoomd.write.GSD.write(state=sim.state, mode='xb', filename=job.fn('random.gsd'))
    job.document['randomized'] = True
    
@flow.FlowProject.operation
@flow.FlowProject.pre.after(rando

Check the status using `python3 project.py status`:

In [2]:
!python3 project.py status --detailed --no-overview -p volume_fraction

Using environment configuration: StandardEnvironment
Fetching labels: 100%|████████████████████████| 3/3 [00:00<00:00, 124583.29it/s]

Detailed View:

job id                            operation          volume_fraction  labels
--------------------------------  ---------------  -----------------  --------
59363805e6f46a715bc154b38dffc4e4  equilibrate [U]                0.6
972b10bd6b308f65f0bc3a06db58cf9d  equilibrate [U]                0.4
c1a59a95a0e8b4526b28cf12aa0a689e  equilibrate [U]                0.5

[U]:unknown [R]:registered [I]:inactive [S]:submitted [H]:held [Q]:queued [A]:active [E]:error




The *equilibrate* step is ready to execute.
On HPC resources you submit **cluster jobs** to the queue so they execute on the compute nodes.
**signac-flow** can automate this process for you with `python3 project.py submit`.
Use the `--pretend` flag first to ensure that the generated **cluster jobs** are correct:

```bash
$ python3 project.py submit --pretend

Submitting cluster job 'octahedra_se/972b10bd6b308f65f0bc3a06db58cf9d/equilibrate/cdc78829a46f27e11ee8a98049bf0575':
 - Group: equilibrate(972b10bd6b308f65f0bc3a06db58cf9d)
# Submit command: sbatch
#!/bin/bash
#SBATCH --job-name="octahedra_se/972b10bd6b308f65f0bc3a06db58cf9d/equilibrate/cdc78829a46f27e11ee8a98049bf0575"
#SBATCH --partition=standard
#SBATCH -t 01:00:00
#SBATCH --nodes=1
#SBATCH --ntasks-per-node=1

set -e
set -u

cd /project/path


# equilibrate(972b10bd6b308f65f0bc3a06db58cf9d)
python3 /project/path/project.py run -o equilibrate -j 972b10bd6b308f65f0bc3a06db58cf9d
# Eligible to run:
# python3 /project/path/project.py exec equilibrate 972b10bd6b308f65f0bc3a06db58cf9d


Submitting cluster job 'octahedra_se/59363805e6f46a715bc154b38dffc4e4/equilibrate/2c15943de4918753dc2373cd33d527ec':
 - Group: equilibrate(59363805e6f46a715bc154b38dffc4e4)
# Submit command: sbatch
#!/bin/bash
#SBATCH --job-name="octahedra_se/59363805e6f46a715bc154b38dffc4e4/equilibrate/2c15943de4918753dc2373cd33d527ec"
#SBATCH --partition=standard
#SBATCH -t 01:00:00
#SBATCH --nodes=1
#SBATCH --ntasks-per-node=1

set -e
set -u

cd /project/path


# equilibrate(59363805e6f46a715bc154b38dffc4e4)
python3 /project/path/project.py run -o equilibrate -j 59363805e6f46a715bc154b38dffc4e4
# Eligible to run:
# python3 /project/path/project.py exec equilibrate 59363805e6f46a715bc154b38dffc4e4


Submitting cluster job 'octahedra_se/c1a59a95a0e8b4526b28cf12aa0a689e/equilibrate/e1ffbf0eafe27af17b2ffc6e0c4c6dd1':
 - Group: equilibrate(c1a59a95a0e8b4526b28cf12aa0a689e)
# Submit command: sbatch
#!/bin/bash
#SBATCH --job-name="octahedra_se/c1a59a95a0e8b4526b28cf12aa0a689e/equilibrate/e1ffbf0eafe27af17b2ffc6e0c4c6dd1"
#SBATCH --partition=standard
#SBATCH -t 01:00:00
#SBATCH --nodes=1
#SBATCH --ntasks-per-node=1

set -e
set -u

cd /project/path


# equilibrate(c1a59a95a0e8b4526b28cf12aa0a689e)
python3 /project/path/project.py run -o equilibrate -j c1a59a95a0e8b4526b28cf12aa0a689e
# Eligible to run:
# python3 /project/path/project.py exec equilibrate c1a59a95a0e8b4526b28cf12aa0a689e
```

In this configuration, **signac flow** submits one **cluster job** for each **signac-job**.

## Partitioning jobs

You can use one **cluster job** for each **signac job** on HPC resources that have shared queues. You need to use a different strategy when your HPC resource schedules jobs *only by full node*, limits the *number of cluster jobs* you can queue at one time, or if you would like to increase throughput when you have many thousands of **signac jobs**.
To accomplish this, use partitions so that one **cluster job** executes many **signac jobs** (see [Parallel Simulations with MPI](../03-Parallel-Simulations-With-MPI/00-index.ipynb) for an introduction to MPI partitions).

This is only a small example.
It sets 2 MPI ranks per **signac job** and executes all three **signac jobs** in one **cluster job**.
In production work you would choose the number of ranks per **signac job** and the number of **signac jobs** per **cluster job** to use an integer number of whole nodes in each **cluster job** with no empty cores or GPUs - for example use 16 ranks per **signac job** and 32 jobs per aggregate to use 4 whole 128-core nodes per **cluster job**.

In [7]:
%pycat project_partitioned.py

import hoomd
import signac
import flow

import math

RANKS_PER_PARTITION = 2
JOBS_PER_AGGREGATE = 3
N_EQUIL_STEPS=200000
WALLTIME_LIMIT=50*60

def create_simulation(job, communicator):
    cpu = hoomd.device.CPU(communicator=communicator)
    sim = hoomd.Simulation(device=cpu, seed=job.statepoint.seed)
    mc = hoomd.hpmc.integrate.ConvexPolyhedron()
    mc.shape['octahedron'] = dict(vertices=[
        (-0.5, 0, 0),
        (0.5, 0, 0),
        (0, -0.5, 0),
        (0, 0.5, 0),
        (0, 0, -0.5),
        (0, 0, 0.5),
    ])
    sim.operations.integrator = mc
    
    return sim

@flow.FlowProject.operation
@flow.FlowProject.pre(lambda job: job.document.get('initialized', False))
@flow.FlowProject.post(lambda job: job.document.get('randomized', False))
def randomize(job):
    sim = create_simulation(job)
    sim.create_state_from_gsd(filename=job.fn('lattice.gsd'))
    sim.run(10e3)
    hoomd.write.GSD.write(state=sim.state, mode='xb', filename=job.fn('random.gsd'))
    job.document

Here are the differences between `project.py` and `project_partitioned.py`:
* The variables `RANKS_PER_PARTITION` and `JOBS_PER_AGGREGATE` set how the **signac jobs** are partitioned into **cluster jobs**.
* `create_simulation` takes the `communicator` argument and passes it to the device constructor.
* The directive `@flow.aggregator.groupsof(num=JOBS_PER_AGGREGATE)` defines `equilibrate` as an aggregate with up to `JOBS_PER_AGGREGATE` **signac jobs** contained in it.
* The pre and post conditions on `equilibrated` are now functions of an array of **signac jobs**:
  ```python
  @flow.FlowProject.pre(lambda *jobs: all('compressed_step' in job.document for job in jobs))
  @flow.FlowProject.post(lambda *jobs: all(equilibrated(job) for job in jobs))
  ```
* The directives set the number of ranks needed by all jobs in the aggregate:
  ```python
    @flow.directives(nranks=lambda *jobs: RANKS_PER_PARTITION * len(jobs), walltime=1)
    ```
* `equilibrate` is now a function of an array of jobs and chooses the job based on the communicator's partition.
  ```python
  def equilibrate(*jobs):
      communicator = hoomd.communicator.Communicator(ranks_per_partition=RANKS_PER_PARTITION)
      job = jobs[communicator.partition]
  ```

Here is the **cluste job** generated by `project_partitioned.py`

```bash
$ python3 project_partitioned.py submit --pretend

Submitting cluster job 'octahedra_se/agg-e202cc8c2ce0bc1ef7a9d9fcdcd62b6d/equilibrate/614e7ec5470deb1e958ac9863ed1fb07':
 - Group: equilibrate(agg-e202cc8c2ce0bc1ef7a9d9fcdcd62b6d)
# Submit command: sbatch
#!/bin/bash
#SBATCH --job-name="octahedra_se/agg-e202cc8c2ce0bc1ef7a9d9fcdcd62b6d/equilibrate/614e7ec5470deb1e958ac9863ed1fb07"
#SBATCH --partition=standard
#SBATCH -t 01:00:00
#SBATCH --nodes=1
#SBATCH --ntasks-per-node=6

set -e
set -u

cd /project/path


# equilibrate(agg-e202cc8c2ce0bc1ef7a9d9fcdcd62b6d)
python3 /project/path/project_partitioned.py run -o equilibrate -j agg-e202cc8c2ce0bc1ef7a9d9fcdcd62b6d
# Eligible to run:
# mpiexec -n 6  python3 /project/path/project_partitioned.py exec equilibrate agg-e202cc8c2ce0bc1ef7a9d9fcdcd62b6d
```

It generates only one **cluster job** that uses MPI to execute the **workflow step** on 6 cores which `equilibrate` splits into 3 partitions that each execute one **signac job** with 2 cores each.

## Summary

In this section of the tutorial, you defined the workflow in a file and used the **signac flow command line interface** to generate **cluster jobs** for submission.
You also learned how to use partitions to more effectively use HPC resources by fully utilizing compute nodes with fewer **cluster jobs**.

This is the end of the tutorial on organizing and executing simulations.

This tutorial only teaches the basics of **signac-flow**.
Read the [signac-flow documentation](http://signac-flow.readthedocs.io/) to learn more.